In [ ]:
%load_ext autoreload
%autoreload 2
import os
opj = os.path.join
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from prfpy_csenf.model import CSenFModel
from prfpy_csenf.stimulus import CSenFStimulus
from prfpy_csenf.fit import CSenFFitter
from prfpy_csenf.rf import csenf_exponential



In [ ]:
# # Load stimulus parameters. 
# # 6 SF levels (sfs_level)
# # 14 contrast levels (con_level)
# sfs_level = np.array([ 0.5,  1.,   3.,   6.,  12.,  18. ] )
# con_level = np.array([ 0.25,     0.38962,  0.60722,  0.94635,  1.4749,   2.2986,   3.5823,   5.583, 8.701,   13.56,    21.134,   32.937,   51.332,   80.     ])

# log_sfs = np.log10(sfs_level)
# contrast_sensitivity = 100 / con_level
# log_SF_grid, CON_S_grid = np.meshgrid(log_sfs, contrast_sensitivity)

# width_r = np.array([  1.400,         1.000,      ])
# sf0     = np.array([  2.000,         6.000,      ])
# maxC    = np.array([100.000,       100.000,      ])
# width_l = np.array([  0.448,         0.448,      ])
# crf_exp = np.array([ 10.000,         3.000,      ])

# csf_mat, csf_curve = csenf_exponential(
#     log_SF_grid = log_SF_grid,
#     CON_S_grid  = CON_S_grid,
#     width_r     = width_r,
#     sf0         = sf0,
#     maxC        = maxC,
#     width_l     = width_l,
#     crf_exp     = crf_exp,
#     return_curve=True,
#     )
# plt.imshow(csf_mat[0,:,:], cmap='magma')    

# csf_mat[0,:,:].squeeze()[9,:]

In [ ]:
# Plot the CSF curve
plt.figure(figsize=(7.5,7.5))
ax = plt.gca()
ax.set_xlabel('Spatial frequency (c/deg)')
ax.set_ylabel('Contrast sensitivity')
ax.set_xscale('log')
ax.set_yscale('log')

# Plot the different CSF curves in test params
for i in range(csf_curve.shape[1]):
    ax.plot(
        sfs_level,
        csf_curve[:,i],
        label=f'{i}',
        linestyle='--', alpha=1, lw=5
        )
# Put a grid on the axis (only the major ones)
ax.grid(which='both', axis='both', linestyle='--', alpha=0.5)
# Make the axis square
ax.set_aspect('equal', 'box') 
ax.set_xticks([.1, 1,10,100])
ax.set_yticks([.1, 1,10,100, 1000])
ax.set_xlim([0.1, 100])
ax.set_ylim([0.1, 1000])
ax.legend()

# Display the matrix
fig, ax = plt.subplots(1,csf_curve.shape[1], figsize=(7,7))
for i in range(csf_curve.shape[1]):
    # Display the matrix, with a colorbar
    im = csf_mat[i,:,:].squeeze()
    ax[i].imshow(im, cmap='magma', vmin=0, vmax=1)            


In [ ]:
        
# Get the timecourses for this model
test_params = {
    'width_r'   :np.array([  1.000,         1.000,      ]),
    'sf0'       :np.array([  1.000,         6.000,      ]),
    'maxC'      :np.array([100.000,       100.000,      ]),
    'width_l'   :np.array([  0.448,         0.448,      ]),
    'crf_exp'   :np.array([  3.000,         3.000,      ]),
    'beta'      :np.array([  1.000,         1.000,      ]),
    'baseline'  :np.array([  0.000,         0.000,      ]),
    'hrf_1'     :np.array([  1.000,         1.000,      ]),
    'hrf_2'     :np.array([  0.000,         0.000,      ]),    
}
test_tc = csenf_model.return_prediction(**test_params)
plt.figure(figsize=(15,7.5))
plt.plot(test_tc.T)    


# Calculate CSF curves + matrix 
csf_mat, csf_curve = csenf_exponential(
    log_SF_grid = csenf_stim.log_SF_grid, 
    CON_S_grid  = csenf_stim.CON_S_grid,
    width_r     = test_params['width_r'], 
    sf0         = test_params['sf0'], 
    maxC        = test_params['maxC'], 
    width_l     = test_params['width_l'], 
    crf_exp     = test_params['crf_exp'],
    return_curve=True,
    )    

# Log CSF curve
logcsf_curve = np.log10(csf_curve)    
logcsf_curve[logcsf_curve<0 ] = 0


# Plot the CSF curve
plt.figure(figsize=(7.5,7.5))
ax = plt.gca()
ax.set_xlabel('Spatial frequency (c/deg)')
ax.set_ylabel('Contrast sensitivity')
ax.set_xscale('log')
ax.set_yscale('log')
# Scatter the points sampled
ax.scatter(
    sfs_seq, 100/con_seq, color='k', alpha=0.5
)
# Plot the different CSF curves in test params
for i in range(test_params['width_r'].size):
    ax.plot(
        sfs_level,
        csf_curve[:,i],
        label=f'{i}',
        linestyle='--', alpha=1, lw=5
        )
# Put a grid on the axis (only the major ones)
ax.grid(which='both', axis='both', linestyle='--', alpha=0.5)
# Make the axis square
ax.set_aspect('equal', 'box') 
ax.set_xticks([.1, 1,10,100])
ax.set_yticks([.1, 1,10,100, 1000])
ax.set_xlim([0.1, 100])
ax.set_ylim([0.1, 1000])
ax.legend()

# Display the matrix
fig, ax = plt.subplots(1,len(test_params['width_r']), figsize=(7,7))
for i in range(test_params['width_r'].size):
    # Display the matrix, with a colorbar
    im = csf_mat[i,:,:].squeeze()
    ax[i].imshow(im, cmap='magma', vmin=0, vmax=1)            
